<a href="https://colab.research.google.com/github/haratimo/EEG_Target2Target/blob/main/BERT_MModel_Parastoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **1**. Load the CSV and Pre-trained BERT model:

In [15]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertModel

# Load your CSV
df = pd.read_csv("622Metaphors_and_W1W2.csv")

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


### 2. Embed the Metaphors using BERT:

In [16]:
def embed_metaphor(metaphor):
    inputs = tokenizer(metaphor, return_tensors="pt", truncation=True, padding=True, max_length=20)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs['last_hidden_state'].mean(dim=1).numpy()

# Using "Metaphor" column to compute embeddings
df["Embedding"] = df["Metaphor"].apply(embed_metaphor)


### 3. Calculate Similarity for Comparative Analysis:

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

def compute_similarity(embedding, reference_embedding):
    return cosine_similarity(embedding.reshape(1, -1), reference_embedding.reshape(1, -1))[0][0]

# For demonstration purposes, using the embedding of the first metaphor as a reference
reference_embedding = df["Embedding"].iloc[0]

df["Similarity_to_Reference"] = df["Embedding"].apply(lambda x: compute_similarity(x, reference_embedding))


### 4. Save the Results:
To save the embeddings and (if computed) similarity scores back to CSV:

In [18]:
# If you want to keep the embeddings in the CSV, you'll need to convert them to a string representation
# as CSVs can't directly store complex data types like lists or arrays.
df["Embedding"] = df["Embedding"].apply(lambda x: ' '.join(map(str, x)))

df.to_csv("MetaphorsW1W2_with_embeddings_622.csv", index=False)


In [19]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

# --- BERT Embedding Function ---
def embed_metaphor(metaphor):
    """
    Embed the given metaphor using BERT.
    """
    inputs = tokenizer(metaphor, return_tensors="pt", truncation=True, padding=True, max_length=20)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs['last_hidden_state'].mean(dim=1).numpy()

# --- Initialize BERT ---
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# --- Load Dataset ---
df = pd.read_csv("MetaphorsW1W2_622.csv")

# --- Calculate Embeddings ---
embeddings = []
for _, row in df.iterrows():
    metaphor = f"{row['W1']} is a {row['W2']}"  # Constructing the metaphor
    embedding = embed_metaphor(metaphor)
    embeddings.append(embedding)

# Saving embeddings to the dataframe
df["BERT_embeddings"] = embeddings

# Assuming you have 'zGoodness' as a column in df, let's compute the cosine similarity
# between each metaphor embedding and average embedding of all metaphors.
avg_embedding = np.mean(embeddings, axis=0)
df["BERT_similarity_avg"] = [cosine_similarity([emb], [avg_embedding])[0][0] for emb in embeddings]

# Compute the correlation between BERT_similarity_avg and zGoodness
correlation = df["BERT_similarity_avg"].corr(df["zGoodness"])
print(f"Correlation between BERT-based similarity (to average) and zGoodness: {correlation:.2f}")

# Saving the updated dataframe with the computed scores
df.to_csv("MetaphorsW1W2_with_scores_622.csv", index=False)


NameError: ignored